# This notebook takes transformed data and aggregates them to produce metrics used for the PowerBI visualization 

In [1]:
import pandas as pd
import numpy as np

import geopandas as gpd
from shapely.geometry import Point

## Read in data

In [ ]:
# File is from 1_data_cleanup_map.ipynb, please run that notebook before proceeding with this one
df = pd.read_csv('20241116-Seattle-new-neighborhoods.csv')

## Data Cleaning and Generate Metrics

In [2]:
# Generate call time, average call time, and time difference
df.loc[df.GEN_ALIAS == 'Magnolia ', 'GEN_ALIAS'] = 'Magnolia'

df['call_time'] = pd.to_datetime(df['call_time'])
df['arrived_time'] = pd.to_datetime(df['arrived_time'])

df['time_difference'] = df['arrived_time'] - df['call_time']
df['time_difference_in_seconds'] = df['time_difference'].dt.total_seconds()


In [4]:
# Filter out Call Types that have no arrival time
# These call types have arrival time of zero, and is skewing the data and the model predictions
df = df.loc[df['Call Type'] != 'ONVIEW']
df = df.loc[df['Call Type'] != 'PROACTIVE (OFFICER INITIATED)']
df = df.loc[df['Call Type'] != 'HISTORY CALL (RETRO)']
df = df.loc[df['Call Type'] != 'IN PERSON COMPLAINT']
df = df.loc[df['Call Type'] != 'SCHEDULED EVENT (RECURRING)']
df['Call Type'].value_counts()


Call Type
911                              2220711
TELEPHONE OTHER, NOT 911         1319131
ALARM CALL (NOT POLICE ALARM)     190106
TEXT MESSAGE                        1299
POLICE (VARDA) ALARM                 158
Name: count, dtype: int64

In [5]:
# Calculate high priority calls (Priority == 1)
high_prio = df.loc[
    (df.Priority == 1.0) 
#   (df.Priority == 2.0) |
#   (df.Priority == 3.0)
]

grouped_high_prio = high_prio.groupby(['call_time_year', 'GEN_ALIAS']).agg(
    avg_call_duration_high_prio=('time_difference_in_seconds', 'mean'), 
    call_volume_high_prio=('time_difference_in_seconds', 'count')
)

res = grouped_high_prio.reset_index()

In [6]:
grouped = df.groupby(['call_time_year', 'GEN_ALIAS']).agg(
    avg_call_duration=('time_difference_in_seconds', 'mean'), 
    call_volume=('time_difference_in_seconds', 'count')
)

res2 = grouped.reset_index()


In [7]:
merged_df = pd.merge(res, res2, on=['call_time_year', 'GEN_ALIAS'])


In [8]:
merged_df

,call_time_year,GEN_ALIAS,avg_call_duration_high_prio,call_volume_high_prio,avg_call_duration,call_volume
0,2009,Ballard,739.243767,361,2780.723277,2685
1,2009,Capitol Hill,371.892977,299,1678.063086,2346
2,2009,Downtown Commercial Core,438.954545,704,1757.567301,4465
3,2009,Duwamish/SODO,539.869048,168,1885.955862,1450
4,2009,First Hill,452.864407,236,1769.071066,1576
...,...,...,...,...,...,...
799,2024,University District,811.091457,995,7520.661550,5472
800,2024,Wallingford,724.749415,427,6394.215379,2809
801,2024,Wedgwood/View Ridge,1000.385542,166,6300.047276,973
802,2024,West Seattle Junction/Genesee Hill,673.728597,549,3667.014872,3362


In [9]:
merged_df.to_csv('aggregated_data_no_pop.csv')

## Add Population Data: add additional population data from an external source into our visualization

In [10]:
# Population data is taken from the 2020 Seattle Census data
pop = pd.read_csv('pop_data_per_year.csv') 
pop

,Year,Population,CRA_ID,Neighborhood Name
0,2020,21377.000000,4.2,Columbia City
1,2019,21209.445381,4.2,Columbia City
2,2018,21041.116450,4.2,Columbia City
3,2017,20791.617045,4.2,Columbia City
4,2016,20543.046186,4.2,Columbia City
...,...,...,...,...
896,2008,5142.218934,13.3,Pioneer Square/International District
897,2021,5940.314400,13.3,Pioneer Square/International District
898,2022,5988.430947,13.3,Pioneer Square/International District
899,2023,6039.931453,13.3,Pioneer Square/International District


In [12]:
pop.rename(columns={'Year': 'call_time_year', 'Neighborhood Name': 'GEN_ALIAS'}, inplace=True)

In [13]:
pop.loc[pop.GEN_ALIAS.isna(), 'GEN_ALIAS'] = 'Missing'

In [15]:
merged = pd.merge(merged_df, pop, on=['call_time_year', 'GEN_ALIAS'], how='left')


In [16]:
merged[
    ['GEN_ALIAS','call_time_year', 'avg_call_duration_high_prio', 'call_volume_high_prio', 'avg_call_duration', 'call_volume', 'Population']
].to_csv('aggregated_data.csv', index=False)


In [17]:
merged = read_csv('aggregated_data.csv')

In [18]:
merged

,GEN_ALIAS,call_time_year,avg_call_duration_high_prio,call_volume_high_prio,avg_call_duration,call_volume,Population
0,Ballard,2009,739.243767,361,2780.723277,2685,9904.416369
1,Capitol Hill,2009,371.892977,299,1678.063086,2346,22464.671870
2,Downtown Commercial Core,2009,438.954545,704,1757.567301,4465,5215.407407
3,Duwamish/SODO,2009,539.869048,168,1885.955862,1450,2847.497626
4,First Hill,2009,452.864407,236,1769.071066,1576,16886.261509
...,...,...,...,...,...,...,...
799,University District,2024,811.091457,995,7520.661550,5472,24262.735243
800,Wallingford,2024,724.749415,427,6394.215379,2809,19491.653256
801,Wedgwood/View Ridge,2024,1000.385542,166,6300.047276,973,17714.515675
802,West Seattle Junction/Genesee Hill,2024,673.728597,549,3667.014872,3362,24295.817502


In [19]:
merged.rename(columns={'GEN_ALIAS':'Neighborhood', 'call_time_year':'Year', 'Population':'population'}, inplace=True)

In [20]:
# Calculating Crime Rate
# dividing the number of reported crimes by the total population. The result is then multiplied by 100,000
merged.loc[merged.population.notna(), 'crime_rate_per_1000'] = (merged['call_volume'] / merged['population']) * 1000

In [22]:
merged.crime_rate_per_1000.max()
merged.crime_rate_per_1000.min()

2788.2265488833746

54.92670631403037

In [23]:
merged.to_csv('seattle_map_aggregated_data.csv')

## add CRA_ID in addition to neighborhood name for PowerBI

In [26]:
pop = pd.read_csv('pop_data_per_year.csv')
df1 = pd.read_csv('seattle_map_aggregated_data.csv')

In [27]:
pop = pop[['CRA_ID', 'Neighborhood Name']].groupby(['CRA_ID', 'Neighborhood Name']).count().reset_index()

In [32]:
merged = df1.merge(pop, left_on='Neighborhood', right_on='Neighborhood Name', how='inner')
merged.shape

(804, 11)

In [33]:
merged.drop('Unnamed: 0', axis=1, inplace=True)

In [34]:
merged

,Neighborhood,Year,avg_call_duration_high_prio,call_volume_high_prio,avg_call_duration,call_volume,population,crime_rate_per_1000,CRA_ID,Neighborhood Name
0,Ballard,2009,739.243767,361,2780.723277,2685,9904.416369,271.091188,10.4,Ballard
1,Ballard,2010,724.524313,473,1947.586670,3796,10025.250249,378.643915,10.4,Ballard
2,Ballard,2011,758.516854,534,2244.921030,3419,10146.555777,336.961633,10.4,Ballard
3,Ballard,2012,944.278878,606,2862.289742,3724,10267.299791,362.704905,10.4,Ballard
4,Ballard,2013,899.820078,767,2670.547731,4473,10394.614308,430.318997,10.4,Ballard
...,...,...,...,...,...,...,...,...,...,...
799,Whittier Heights,2020,731.389831,295,3890.047259,1587,14778.000000,107.389363,10.2,Whittier Heights
800,Whittier Heights,2021,825.009375,320,5447.408151,1693,14899.179600,113.630418,10.2,Whittier Heights
801,Whittier Heights,2022,818.709970,331,5076.280022,1807,15019.862955,120.307356,10.2,Whittier Heights
802,Whittier Heights,2023,1155.288835,412,7397.429206,1801,15149.033776,118.885470,10.2,Whittier Heights


In [36]:
merged.to_csv('seattle_map_aggregated_data_withCRA_ID.csv')
# this file is used to generate the PowerBI metrics

In [ ]:
'done'